In [ ]:
# !conda install -y -c conda-forge pyspark
# !conda install -y -c conda-forge black
# !conda install -y -c conda-forge pip
# !pip install delta-spark==2.1.1
# !sudo wget --user jovyan --password jovyan https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.1.0/delta-core_2.12-2.1.0.jar -P $SPARK_HOME/jars/
# !conda install -y -c conda-forge pandera
# !conda install -y -c conda-forge pandera[pyspark]
# !conda install -y -c conda-forge cerberus

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, DataFrame
from pyspark.conf import SparkConf
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from functools import reduce
import json
import glob
import csv
from delta import *
import datetime

builder = (
    SparkSession.builder.master("local[*]")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

# spark = SparkSession.builder \
#     .master("local") \
#     .getOrCreate()

In [2]:
all_files = all_files = glob.glob("../data/SMES_IT_ES_FR/SMESFR102024100220206/*.csv")
all_files

['../data/SMES_IT_ES_FR/SMESFR102024100220206/SMESFR102024100220206_2020_10_31_Amortization_Profile_6bc25045-00da-419f-be8e-26c6bbecd4c2.csv',
 '../data/SMES_IT_ES_FR/SMESFR102024100220206/SMESFR102024100220206_2020_10_31_Bond_Info_6bc25045-00da-419f-be8e-26c6bbecd4c2.csv',
 '../data/SMES_IT_ES_FR/SMESFR102024100220206/SMESFR102024100220206_2020_10_31_Collateral_6bc25045-00da-419f-be8e-26c6bbecd4c2.csv',
 '../data/SMES_IT_ES_FR/SMESFR102024100220206/SMESFR102024100220206_2020_10_31_Loan_Data_6bc25045-00da-419f-be8e-26c6bbecd4c2.csv',
 '../data/SMES_IT_ES_FR/SMESFR102024100220206/SMESFR102024100220206_2020_12_31_Amortization_Profile_f84efe9c-97db-4590-bcdb-67ae674b55bd.csv',
 '../data/SMES_IT_ES_FR/SMESFR102024100220206/SMESFR102024100220206_2020_12_31_Bond_Info_f84efe9c-97db-4590-bcdb-67ae674b55bd.csv',
 '../data/SMES_IT_ES_FR/SMESFR102024100220206/SMESFR102024100220206_2020_12_31_Collateral_f84efe9c-97db-4590-bcdb-67ae674b55bd.csv',
 '../data/SMES_IT_ES_FR/SMESFR102024100220206/SMESFR

### Pandera

In [ ]:
for f_name in all_files:
    content =[]
    if "Loan_Data" in f_name:
        with open(f_name, "r") as f:
            for i, line in enumerate(csv.reader(f)):
                curr_line = line
                if i == 0:
                    col_names = curr_line
                    col_names[0] = "AS1"
                elif i == 1:
                    continue
                else:
                    if len(curr_line) == 0:
                        continue
                    content.append(curr_line)
                    # repartition = 4 instances * 8 cores each * 3 for replication factor
            df = spark.createDataFrame(content, col_names).repartition(96)
            break
df.printSchema()

In [ ]:
import pandera as pa
import datetime
from pandera import (
    Column, DataFrameSchema, Index, SeriesSchema, Bool, Category, Check,
    DateTime, Float, Int, Object, String, Timedelta, errors, Decimal)

def check_isoformat(dt):
    try:
        datetime.datet.isoformat(dt)
        return True
    except:
        return False

assets_loans_schema = DataFrameSchema(
    {'AS1':Column(DateTime,Check(lambda dt: check_isoformat(dt))),
        'AS2':Column(String),
        # 'AS3':Column(String),
        # 'AS4':Column(String),
        # 'AS5':Column(String),
        # 'AS6':Column(String),
        # 'AS7':Column(String),
        # 'AS8':Column(String),
        # 'AS15':Column(String),
        # 'AS16':Column(String),
        # 'AS17':Column(String),
        # 'AS18':Column(String),
        # 'AS19':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        'AS19':Column(DateTime,[Check.greater_than_or_equal_to("2012-01-01") ,Check.less_than_or_equal_to("2030-12-31")],nullable=True),
        # 'AS19':Column(DateTime,Check.in_range("2012-01-01","2030-12-31"),nullable=True),
        # 'AS20':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS21':Column(String),
        # 'AS22':Column(String),
        # 'AS23':Column(Bool, Check(lambda b: b.isin(["Y","N"]) | b.startswith("ND"))),
        # 'AS24':Column(String),
        # 'AS25':Column(String),
        # 'AS26':Column(String),
        # 'AS27':Column(String),
        # 'AS28':Column(Float),
        # 'AS29':Column(Bool,Check(lambda b: b.isin(["Y","N"]) | b.startswith("ND")), nullable=True),
        'AS30':Column(Int),
        # 'AS31':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS32':Column(String),
        # 'AS33':Column(String),
        # 'AS34':Column(String),
        # 'AS35':Column(String),
        # 'AS36':Column(String),
        # 'AS37':Column(Float),
        # 'AS38':Column(Float),
        # 'AS39':Column(Float),
        # 'AS40':Column(Float),
        # 'AS41':Column(Float),
        # 'AS42':Column(String),
        # 'AS43':Column(String),
        # 'AS44':Column(Float),
        # 'AS45':Column(String),
        # 'AS50':Column(DateTime,Check(lambda dt: check_isoformat(dt))),
        # 'AS51':Column(DateTime,Check(lambda dt: check_isoformat(dt))),
        # 'AS52':Column(String),
        # 'AS53':Column(Bool,Check(lambda b: b.isin(["Y","N"]) | b.startswith("ND"))),
        # 'AS54':Column(Float),
        # 'AS55':Column(Float),
        # 'AS56':Column(Float),
        # 'AS57':Column(String),
        # 'AS58':Column(String),
        # 'AS59':Column(String),
        # 'AS60':Column(Float),
        # 'AS61':Column(Float),
        # 'AS62':Column(String),
        # 'AS63':Column(Float),
        # 'AS64':Column(Float),
        # 'AS65':Column(String),
        # 'AS66':Column(Float),
        # 'AS67':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS68':Column(String),
        # 'AS69':Column(Float),
        # 'AS70':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS71':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS80':Column(Float),
        # 'AS81':Column(Float),
        # 'AS82':Column(Float),
        # 'AS83':Column(String),
        # 'AS84':Column(String),
        # 'AS85':Column(Float),
        # 'AS86':Column(Float),
        # 'AS87':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS88':Column(Float),
        # 'AS89':Column(String),
        # 'AS90':Column(Float),
        # 'AS91':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS92':Column(String),
        # 'AS93':Column(Float),
        # 'AS94':Column(String),
        # 'AS100':Column(Float),
        # 'AS101':Column(Float),
        # 'AS102':Column(Float),
        # 'AS103':Column(Float),
        # 'AS104':Column(Float),
        # 'AS105':Column(Float),
        # 'AS106':Column(Float),
        # 'AS107':Column(Float),
        # 'AS108':Column(Float),
        # 'AS109':Column(Float),
        # 'AS110':Column(Float),
        # 'AS111':Column(String),
        # 'AS112':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS115':Column(Float),
        # 'AS116':Column(Float),
        # 'AS117':Column(Float),
        # 'AS118':Column(Float),
        # 'AS119':Column(Float),
        # 'AS120':Column(Float),
        # 'AS121':Column(Bool,Check(lambda b: b.isin(["Y","N"]) | b.startswith("ND"))),
        # 'AS122':Column(Bool,Check(lambda b: b.isin(["Y","N"]) | b.startswith("ND"))),
        # 'AS123':Column(String),
        # 'AS124':Column(DateTime,Check(lambda dt: check_isoformat(dt))),
        # 'AS125':Column(Float),
        # 'AS126':Column(Float),
        # 'AS127':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS128':Column(Float),
        # 'AS129':Column(String),
        # 'AS130':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS131':Column(Bool,[Check(lambda b: b.isin(["Y","N"]) | b.startswith("ND")),Check(lambda b: b.startswith("ND"))], nullable=True),
        # 'AS132':Column(Float),
        # 'AS133':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS134':Column(DateTime,Check(lambda dt: check_isoformat(dt))),
        # 'AS135':Column(Float),
        # 'AS136':Column(Float),
        # 'AS137':Column(DateTime,Check(lambda dt: check_isoformat(dt)),nullable=True),
        # 'AS138':Column(Float),
    },
    unique=["AS1","AS3"],
    # strict=True,
    coerce=True,
)

assets_loans_schema.validate(df.toPandas())

### Cerberus

In [8]:
for f_name in all_files:
    content =[]
    if "Loan_Data" in f_name:
        with open(f_name, "r") as f:
            for i, line in enumerate(csv.reader(f)):
                if i == 0:
                    col_names = line
                    col_names[0] = "AS1"
                elif i == 1:
                    continue
                else:
                    if len(line) == 0:
                        continue
                    clean_line = [None if (el == '') or (el.startswith("ND")) else el for el in line]
                    content.append({col_names[i]: clean_line[i] for i in range(len(clean_line))})
            break
content

[{'AS1': '2020-10-31',
  'AS2': 'FCT Bpifrance SME 2020-1',
  'AS3': 'DOS000572500/6',
  'AS4': 'BPI FRANCE',
  'AS5': '549300Q05CR26H4X5294',
  'AS6': 'BPI FRANCE',
  'AS7': '3835847',
  'AS8': None,
  'AS15': 'FR',
  'AS16': 'FRF11',
  'AS17': None,
  'AS18': '2',
  'AS19': None,
  'AS20': None,
  'AS21': None,
  'AS22': '2',
  'AS23': 'N',
  'AS24': None,
  'AS25': '1',
  'AS26': '1',
  'AS27': None,
  'AS28': None,
  'AS29': None,
  'AS30': None,
  'AS31': None,
  'AS32': None,
  'AS33': None,
  'AS34': None,
  'AS35': None,
  'AS36': None,
  'AS37': '99.99',
  'AS38': None,
  'AS39': None,
  'AS40': None,
  'AS41': None,
  'AS42': 'G46.6.9',
  'AS43': None,
  'AS44': None,
  'AS45': None,
  'AS50': '2014-09',
  'AS51': '2022-03',
  'AS52': 'EUR',
  'AS53': 'N',
  'AS54': '100000.00',
  'AS55': '30000.00',
  'AS56': '30000.00',
  'AS57': None,
  'AS58': '2',
  'AS59': '2',
  'AS60': None,
  'AS61': None,
  'AS62': '1',
  'AS63': None,
  'AS64': None,
  'AS65': '1',
  'AS66': '0.00'

In [15]:
from cerberus import Validator

def run_data_profiling(event, validator_schema):
    """Perform data profiling with Cerberus package.

    :param event: nested dictionary representation of the dvault.
    :param validator_schema: JSON schema to be use to validate dvault.
    :return bool: data profiling flag when is successful/unsuccessful.
    :return errors: dictionaries of exceptions encoutered in the profiling.
    """
    v = Validator(validator_schema, allow_unknown=True)
    flag = v.validate(event)
    if not flag:
        return (flag, v.errors)
    return (flag, {})

to_date = lambda s: datetime.datetime.strptime(s, '%Y-%m-%d')
to_number= lambda n: float(n)

assets_loans_schema = {
    "AS1":{
        "type":"datetime",
        "required": True,
        "coerce": to_date,
        "min": datetime.datetime(2012, 1, 1),
        "max": datetime.datetime(2030, 12, 31)
    },
    "AS2": {
         "type":"string",
        "required": True,
    },
    "AS19": {
         "type":"datetime",
        "required": True,
        "coerce": to_date,
        "min": datetime.datetime(2012, 1, 1),
        "max": datetime.datetime(2030, 12, 31),
        "nullable":True
    },
    "AS30": {
        "type": "number",
        "required": True,
        "nullable": True,
        "coerce": to_number,
        "min": 0.0
    }
}

print(content[0])
run_data_profiling(content[0], assets_loans_schema)

{'AS1': '2020-10-31', 'AS2': 'FCT Bpifrance SME 2020-1', 'AS3': 'DOS000572500/6', 'AS4': 'BPI FRANCE', 'AS5': '549300Q05CR26H4X5294', 'AS6': 'BPI FRANCE', 'AS7': '3835847', 'AS8': None, 'AS15': 'FR', 'AS16': 'FRF11', 'AS17': None, 'AS18': '2', 'AS19': None, 'AS20': None, 'AS21': None, 'AS22': '2', 'AS23': 'N', 'AS24': None, 'AS25': '1', 'AS26': '1', 'AS27': None, 'AS28': None, 'AS29': None, 'AS30': None, 'AS31': None, 'AS32': None, 'AS33': None, 'AS34': None, 'AS35': None, 'AS36': None, 'AS37': '99.99', 'AS38': None, 'AS39': None, 'AS40': None, 'AS41': None, 'AS42': 'G46.6.9', 'AS43': None, 'AS44': None, 'AS45': None, 'AS50': '2014-09', 'AS51': '2022-03', 'AS52': 'EUR', 'AS53': 'N', 'AS54': '100000.00', 'AS55': '30000.00', 'AS56': '30000.00', 'AS57': None, 'AS58': '2', 'AS59': '2', 'AS60': None, 'AS61': None, 'AS62': '1', 'AS63': None, 'AS64': None, 'AS65': '1', 'AS66': '0.00', 'AS67': None, 'AS68': '1', 'AS69': None, 'AS70': None, 'AS71': None, 'AS80': '4.19000000', 'AS81': None, 'AS8

(True, {})

In [12]:
v = Validator(assets_loans_schema, allow_unknown=True)
normalized_document = v.normalized(content[0], assets_loans_schema)
normalized_document

{'AS1': datetime.datetime(2020, 10, 31, 0, 0),
 'AS2': 'FCT Bpifrance SME 2020-1',
 'AS3': 'DOS000572500/6',
 'AS4': 'BPI FRANCE',
 'AS5': '549300Q05CR26H4X5294',
 'AS6': 'BPI FRANCE',
 'AS7': '3835847',
 'AS8': None,
 'AS15': 'FR',
 'AS16': 'FRF11',
 'AS17': None,
 'AS18': '2',
 'AS19': None,
 'AS20': None,
 'AS21': None,
 'AS22': '2',
 'AS23': 'N',
 'AS24': None,
 'AS25': '1',
 'AS26': '1',
 'AS27': None,
 'AS28': None,
 'AS29': None,
 'AS30': None,
 'AS31': None,
 'AS32': None,
 'AS33': None,
 'AS34': None,
 'AS35': None,
 'AS36': None,
 'AS37': '99.99',
 'AS38': None,
 'AS39': None,
 'AS40': None,
 'AS41': None,
 'AS42': 'G46.6.9',
 'AS43': None,
 'AS44': None,
 'AS45': None,
 'AS50': '2014-09',
 'AS51': '2022-03',
 'AS52': 'EUR',
 'AS53': 'N',
 'AS54': '100000.00',
 'AS55': '30000.00',
 'AS56': '30000.00',
 'AS57': None,
 'AS58': '2',
 'AS59': '2',
 'AS60': None,
 'AS61': None,
 'AS62': '1',
 'AS63': None,
 'AS64': None,
 'AS65': '1',
 'AS66': '0.00',
 'AS67': None,
 'AS68': '1',
